In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, SimpleRNN
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

2025-05-13 11:30:36.810281: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-13 11:30:37.405732: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-13 11:30:37.523639: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-13 11:30:40.398413: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; 

In [2]:
#loadig the imdb dataset

max_features = 10000 #define the vocabulary size

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features) 

print(f" training data shape:{X_train.shape}, Testing data shape: {X_test.shape}")
print(f" training data shape:{y_train.shape}, Testing data shape: {y_test.shape}")

 training data shape:(25000,), Testing data shape: (25000,)
 training data shape:(25000,), Testing data shape: (25000,)


In [3]:
#sample review and its label
sample_review = X_train[0]
sample_label = y_train[0]

In [4]:
#mapping of index to their respective words tin understand the sentence

word_index = imdb.get_word_index()
reversed_word_index = {value:key for key,value in word_index.items()}

In [5]:
decoded_sample_review = " ".join([reversed_word_index.get(i-3, "?") for i in sample_review])
decoded_sample_review

"? this film was just brilliant casting location scenery story direction everyone's really suited the part they played and you could just imagine being there robert ? is an amazing actor and now the same being director ? father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for ? and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also ? to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the ? list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you th

In [6]:
max_len = 500

X_train = sequence.pad_sequences(X_train, maxlen=max_len)
X_test = sequence.pad_sequences(X_test, maxlen= max_len)
X_train

array([[   0,    0,    0, ...,   19,  178,   32],
       [   0,    0,    0, ...,   16,  145,   95],
       [   0,    0,    0, ...,    7,  129,  113],
       ...,
       [   0,    0,    0, ...,    4, 3586,    2],
       [   0,    0,    0, ...,   12,    9,   23],
       [   0,    0,    0, ...,  204,  131,    9]], dtype=int32)

In [7]:
#creating the simple rnn model
dim = 128  #feature representation 
model = Sequential()
model.add(Embedding(max_features,dim,input_length = max_len))  #Embedding layers
model.add(SimpleRNN(128, activation='relu'))
model.add(Dense(1,activation="sigmoid"))

model.compile(optimizer = 'adam', metrics = ["accuracy"],loss="binary_crossentropy")

2025-05-13 11:30:56.809028: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2025-05-13 11:30:56.809108: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: archana-Katana-GF76-11UE
2025-05-13 11:30:56.809128: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: archana-Katana-GF76-11UE
2025-05-13 11:30:56.809378: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 550.144.3
2025-05-13 11:30:56.809432: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 550.144.3
2025-05-13 11:30:56.809448: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 550.144.3
2025-05-13 11:30:56.811633: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU inst

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               32896     
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,313,025
Trainable params: 1,313,025
Non-trainable params: 0
_________________________________________________________________


In [9]:
early_stopping  = EarlyStopping(patience=5,monitor="val_loss", restore_best_weights=True)

log_dir = "logs/fit"
tensorboard =  TensorBoard(log_dir=log_dir, histogram_freq=1)

In [10]:
#training the model 
history = model.fit(
    X_train,y_train, epochs=10,batch_size=32,
    validation_data=(X_test,y_test),
    callbacks =[early_stopping]
)

Epoch 1/10
782/782 [==============================] - 277s 351ms/step - loss: 3617172.2500 - accuracy: 0.6344 - val_loss: 0.6332 - val_accuracy: 0.6226
Epoch 2/10
782/782 [==============================] - 273s 349ms/step - loss: 0.5576 - accuracy: 0.7082 - val_loss: 0.5307 - val_accuracy: 0.7457
Epoch 3/10
782/782 [==============================] - 269s 344ms/step - loss: 28.5616 - accuracy: 0.7996 - val_loss: 0.4647 - val_accuracy: 0.7883
Epoch 4/10
782/782 [==============================] - 271s 347ms/step - loss: 73496633344.0000 - accuracy: 0.7763 - val_loss: 0.6696 - val_accuracy: 0.5720
Epoch 5/10
782/782 [==============================] - 264s 338ms/step - loss: 0.4871 - accuracy: 0.7522 - val_loss: 0.4518 - val_accuracy: 0.7954
Epoch 6/10
782/782 [==============================] - 268s 343ms/step - loss: 4357327.5000 - accuracy: 0.7302 - val_loss: 0.6178 - val_accuracy: 0.6494
Epoch 7/10
782/782 [==============================] - 267s 342ms/step - loss: 0.4948 - accuracy: 0.74

In [11]:
model.save("simple_rnn.h5")